In [1]:
import pandas as pd
import cv2
import glob
import os
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
fruits_name = [name for name in os.listdir(r'D:\Machine Learning 3\fruits-360\Training')]

In [3]:
# make an empty list to store training image and label
trainData = []
trainLabels = []

# loop through images in training folder, read image and append to dataframe
for name in fruits_name:
    folder_path = r'D:\Machine Learning 3\fruits-360\Training\\' + name + '\\'
    glob_path = glob.glob(folder_path + '*.jpg')
    for im_path in glob_path:
        im_tempo = cv2.imread(im_path)
        trainData.append(im_tempo)
        trainLabels.append(name)
trainData = np.array(trainData)
trainLabels = np.array(trainLabels)

In [4]:
trainData.shape

(41322, 100, 100, 3)

In [5]:
# make an empty list to store test image and label
testData = []
testLabels = []

# loop through images in test folder, read image and append to dataframe
for name in fruits_name:
    folder_path = r'D:\Machine Learning 3\fruits-360\Test\\' + name + '\\'
    glob_path = glob.glob(folder_path + '*.jpg')
    for im_path in glob_path:
        im_tempo = cv2.imread(im_path)
        testData.append(im_tempo)
        testLabels.append(name)
testData = np.array(testData)
testLabels = np.array(testLabels)

In [6]:
testData.shape

(13877, 100, 100, 3)

In [7]:
from LeNet import LeNet

In [8]:
# scale data to [0,1]
trainData = trainData.astype('float32')/255.0

In [9]:
# factorize labels and keep a save record for later reference
tempo = np.concatenate((trainLabels, testLabels))                          # concatenate labels into one
factorize = pd.factorize(tempo)                                            # factorzie labels
tempo_category = np_utils.to_categorical(factorize[0], len(fruits_name))   # categorize labels
trainLabels = tempo_category[:len(trainLabels)]                            # put categorized labels to data
testLabels = tempo_category[len(trainLabels):]                             # put categorized labels to data_test
fruit_table = pd.Series(factorize[1])                                      # create table for later references
del tempo
del tempo_category

In [10]:
from keras.optimizers import Adagrad

In [11]:
# optimizer: SGD
opt = Adagrad(lr=0.01)

In [12]:
model = LeNet.build(numChannels=3, imgRows=100, imgCols=100, numClasses=len(fruits_name),
                    weightsPath = None)

In [13]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 6)       456       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 6)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 16)        2416      
_________________________________________________________________
activation_2 (Activation)    (None, 50, 50, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
__________

In [15]:
model_history = model.fit(trainData, trainLabels, batch_size = 128, epochs = 10, verbose = 1)

Epoch 1/10
41322/41322 [==============================] - 371s 9ms/step - loss: 0.8319 - acc: 0.8028
Epoch 2/10
41322/41322 [==============================] - 369s 9ms/step - loss: 0.0381 - acc: 0.9921
Epoch 3/10
41322/41322 [==============================] - 365s 9ms/step - loss: 0.0225 - acc: 0.9943
Epoch 4/10
41322/41322 [==============================] - 369s 9ms/step - loss: 0.0173 - acc: 0.9947
Epoch 5/10
41322/41322 [==============================] - 370s 9ms/step - loss: 0.0155 - acc: 0.9947
Epoch 6/10
41322/41322 [==============================] - 369s 9ms/step - loss: 0.0138 - acc: 0.9947
Epoch 7/10
41322/41322 [==============================] - 371s 9ms/step - loss: 0.0140 - acc: 0.9944
Epoch 8/10
41322/41322 [==============================] - 371s 9ms/step - loss: 0.0126 - acc: 0.9945
Epoch 9/10
41322/41322 [==============================] - 430s 10ms/step - loss: 0.0121 - acc: 0.9950
Epoch 10/10
41322/41322 [==============================] - 492s 12ms/step - loss: 0.0111 -

In [16]:
# save model
model.save_weights(r'D:\Machine Learning 3\Assignment fruit\Benchmark\Variation\Tuned\Adagrad.hdf5')

In [17]:
# save training history
import pickle
with open(r'D:\Machine Learning 3\Assignment fruit\Benchmark\Variation\Tuned\Adagrad-history', 'wb') as fileop:
    pickle.dump(model_history, fileop)

In [18]:
# delete trainData
del trainData

In [19]:
# scale testData to [0,1]
testData = testData.astype('float32')/255.0

In [20]:
# predict on test dataset
predict_label = model.predict(testData, batch_size = 128, verbose = 1)

13877/13877 [==============================] - 84s 6ms/step


In [21]:
# turn the predict vector to final predict base on argmax
predict_label = [np.argmax(i) for i in predict_label]

In [22]:
# turn the test labels vector to final label base on argmax
testLabels = [np.argmax(i) for i in testLabels]

In [23]:
from sklearn.metrics import classification_report

In [24]:
# classification report
print(classification_report(testLabels, predict_label))

              precision    recall  f1-score   support

           0       0.95      0.74      0.83       164
           1       1.00      0.99      1.00       164
           2       1.00      1.00      1.00       164
           3       0.77      0.98      0.86       161
           4       0.98      0.99      0.98       164
           5       0.87      0.79      0.83       164
           6       1.00      0.96      0.98       164
           7       0.81      1.00      0.89       144
           8       1.00      1.00      1.00       166
           9       1.00      0.89      0.94       164
          10       1.00      0.94      0.97       164
          11       1.00      1.00      1.00       143
          12       1.00      1.00      1.00       166
          13       1.00      0.80      0.89       166
          14       0.96      0.74      0.84       166
          15       0.83      1.00      0.91       166
          16       1.00      1.00      1.00       164
          17       1.00    